The code below connects to the database containing the Chicago traffic dataset entries, and queries the information to extract the number of vehicles at each incoming lane for 7 days. For each day, 144 route files are created to account for each 10-minute interval in the 24-hour cycle, and each route file includes route definitions and vehicle elements. The number of vehicle elements originating from an incoming lane during a paritcular time interval on a specifc day depends on the values queried from the table. Those values are iterated in order to instantiate the appropriate number of vehicles.
 This ensures that the PPO model is trained with sufficient real-world Chicago traffic data. 

 Please run this code to generate the training route files.

In [1]:
import xml.etree.ElementTree as ET
import random
import sqlite3

conn = sqlite3.connect('traffic_interval_data.db') #Connect to the database
cursor = conn.cursor()

vehicle_numbers_dict = {}

for i in range(1, 145):
       vehicle_numbers_dict[i] = [[],[],[],[],[],[],[]] #Initialize lists to store vehicle numbers for 7 days for EACH 10-minute interval

results = cursor.execute("SELECT message_count, segment_id, time_indices FROM chicago_traffic_dataset_table").fetchall()
for time_index in range(1, 145): 
    results_per_time_index = list(filter(lambda result: int(result[2]) == time_index, results))
    for segment in [1240, 455, 1203, 448, 1238, 274, 1201, 258]:
        results_per_segment = list(filter(lambda result: int(result[1]) == segment, results_per_time_index))[0:7]
        for day_index in range(0, 7):
          vehicle_numbers_dict[time_index][day_index].append(2 + int(results_per_segment[day_index][0]))

for time_index in range(1, 145):
  for day_index in range(0, 7):
      vehicle_numbers = vehicle_numbers_dict[time_index][day_index]
      print(vehicle_numbers)
      route = ET.Element("routes")
      route.append(ET.Element('route', id='route01', edges = '435326187#0 435677953#0 -435677953#2 377882482#0'))
      route.append(ET.Element('route', id='route02', edges = "-377882482#1 435677953#0 57782117#0 -57782117#3 435677955#0"))
      route.append(ET.Element('route', id='route03', edges = "-435677953#2 377882482#0"))
      route.append(ET.Element('route', id='route04', edges = "-435326192#1 435677953#0 -435677953#2 -435326187#1"))
      route.append(ET.Element('route', id='route05', edges = "435677953#0 57782117#0 -57782117#3 -1303785156#8"))
      route.append(ET.Element('route', id='route06', edges = "1303785156#0 -435677953#2 435326192#0"))
      route.append(ET.Element('route', id='route07', edges = '-435677955#1 -1303785156#8 1303785156#0 57782117#0'))
      route.append(ET.Element('route', id='route08', edges = "-57782117#3 -1303785156#8 1303785156#0 -435677953#2"))
      route.append(ET.Element('vType', id = 'taxi', length = "5.0", minGap = "1.0", maxSpeed="16.67", decel="2.1")) #setting taxi vehicle type for later reference

      index = 0
      departTimes = []
      departTimeToRouteAttr = {}
      adjust = 0.00001
      for number in vehicle_numbers:
         for counter in range(0, number):
          index+=1
          departTime = random.randint(0, 200)
          while departTime in departTimes:
            departTime+=adjust
          departTimes.append(departTime)
          departTimeToRouteAttr[departTime] = [f'{index}', f'route0{vehicle_numbers.index(number)+ 1}'] 
        
      departTimes.sort(reverse = False, key = None) #sort depart times in ascending order to abide by the rules of SUMO route files 

      for departTime in departTimes:
        route.append(ET.Element('vehicle', id = departTimeToRouteAttr[departTime][0], type="taxi", route=departTimeToRouteAttr[departTime][1], depart = str(round(departTime, 2))))
      ET.ElementTree(route).write(f'new_sumo_route-file_at_time_index_{time_index}_for_day_{day_index}.rou.xml')





[13, 12, 7, 12, 2, 6, 2, 2]
[5, 13, 2, 13, 2, 2, 2, 12]
[2, 15, 2, 2, 2, 2, 2, 20]
[2, 15, 2, 16, 2, 5, 2, 14]
[2, 12, 2, 12, 2, 2, 2, 10]
[2, 12, 2, 15, 11, 2, 2, 22]
[2, 2, 10, 2, 2, 4, 2, 2]
[2, 22, 6, 2, 2, 20, 2, 11]
[12, 14, 15, 12, 2, 15, 7, 3]
[11, 14, 14, 11, 2, 8, 12, 23]
[9, 14, 11, 17, 2, 10, 13, 14]
[10, 8, 14, 15, 2, 12, 6, 18]
[8, 12, 7, 13, 2, 2, 2, 4]
[2, 9, 2, 7, 2, 15, 2, 19]
[3, 13, 11, 27, 2, 2, 15, 3]
[2, 24, 2, 7, 2, 2, 2, 11]
[14, 13, 2, 9, 2, 2, 2, 2]
[5, 12, 6, 11, 3, 3, 2, 3]
[2, 23, 2, 15, 2, 2, 6, 2]
[13, 27, 2, 13, 2, 2, 2, 5]
[12, 12, 12, 22, 2, 2, 3, 2]
[2, 2, 2, 10, 2, 20, 2, 14]
[2, 13, 9, 19, 2, 20, 2, 12]
[2, 13, 12, 14, 2, 21, 8, 13]
[21, 13, 12, 29, 2, 21, 15, 11]
[12, 12, 11, 16, 2, 3, 2, 13]
[2, 12, 13, 32, 2, 19, 15, 10]
[2, 8, 2, 19, 2, 24, 2, 13]
[10, 13, 2, 23, 2, 2, 15, 2]
[11, 4, 2, 6, 2, 2, 5, 2]
[2, 13, 7, 24, 2, 2, 2, 2]
[2, 18, 6, 7, 2, 2, 6, 2]
[4, 12, 7, 22, 2, 11, 2, 2]
[9, 12, 8, 8, 2, 2, 2, 2]
[13, 2, 17, 6, 2, 2, 8, 2]
[12, 11, 2,